# 如何检查可运行项
:::info 前提条件
本指南假定您熟悉以下概念：- [LangChain 表达式语言 (LCEL)](/docs/concepts/lcel)- [链式运行](/docs/how_to/sequence/)
:::
一旦你使用 [LangChain 表达式语言](/docs/concepts/lcel) 创建了一个可运行对象，通常你会希望检查它以更好地理解其运行机制。本笔记本介绍了一些实现这一目的的方法。
本指南展示了几种以编程方式检查链内部步骤的方法。如果您更关注如何调试链中的问题，请参阅[此章节](/docs/how_to/debugging)。
首先，让我们创建一个示例链。我们将创建一个用于检索的链：

In [ ]:
%pip install -qU langchain langchain-openai faiss-cpu tiktoken

In [2]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

## 获取图表
你可以使用 `get_graph()` 方法来获取可运行对象的图形表示：

In [ ]:
chain.get_graph()

## 打印图表
虽然这不够清晰易读，但你可以使用 `print_ascii()` 方法以更易懂的方式展示该图表：

In [5]:
chain.get_graph().print_ascii()

           +---------------------------------+         
           | Parallel<context,question>Input |         
           +---------------------------------+         
                    **               **                
                 ***                   ***             
               **                         **           
+----------------------+              +-------------+  
| VectorStoreRetriever |              | Passthrough |  
+----------------------+              +-------------+  
                    **               **                
                      ***         ***                  
                         **     **                     
           +----------------------------------+        
           | Parallel<context,question>Output |        
           +----------------------------------+        
                             *                         
                             *                         
                             *                  

## 获取提示
你可能想通过 `get_prompts()` 方法查看链式调用中使用的提示词：

In [6]:
chain.get_prompts()

[ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))])]

## 后续步骤
你现在已经学会了如何内省你组合的LCEL链。
接下来，请查阅本节中关于可运行项的其他操作指南，或参考相关的[链式调试操作指南](/docs/how_to/debugging)。